In [1]:
from __future__ import print_function

import sys
import os
import time

import numpy as np
import theano
import theano.tensor as T

import lasagne

import pandas 

from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
def load_dataset():

#     data = pandas.read_csv('train.tsv', sep = '\t') 
#     testdata = pandas.read_csv('test.tsv', sep = '\t') 
    import pickle
    f = open('rotten_tomatoes_train.pickle')
    X = pickle.load(f)
    y = pickle.load(f)
    f.close()
    
    # We can now download and read the training and test set images and labels.
#     X = data.Phrase
#     y = data.Sentiment
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5, random_state=1)

    # We reserve the last 10000 training examples for validation.
    X_train, X_val = X_train[:-10000], X_train[-10000:]
    y_train, y_val = y_train[:-10000], y_train[-10000:]

    # We just return all the arrays in order, as expected in main().
    # (It doesn't matter how we do this as long as we can read them again.)
    
#     vectorizer = CountVectorizer(stop_words='english')
#     X_train = vectorizer.fit_transform([r for r in X_train])
#     X_test = vectorizer.transform([r for r in X_test])
#     X_val = vectorizer.transform([r for r in X_val])
    
    return np.asarray(X_train), np.asarray(y_train), np.asarray(X_val), np.asarray(y_val), np.asarray(X_test), np.asarray(y_test)

In [45]:
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
y_val = y_val.astype(np.int32)

In [57]:
print (X_train[1,:])
print (y_train)

[ 0.03379533 -0.01067933 -0.001843    0.04078133 -0.05101033  0.04132
  0.042465    0.01693667  0.02975167  0.06493433  0.04356933 -0.008447
 -0.004513    0.01929833 -0.05798567  0.05939867 -0.045535    0.09794233
 -0.07398533 -0.078295   -0.04083433  0.00614867  0.05273967  0.00309433
  0.01245433 -0.00722533 -0.014074    0.05410367  0.01350467 -0.00134367
 -0.107147   -0.065084    0.03507267  0.020408    0.00586667  0.01339233
  0.08467633  0.01912833 -0.04472767  0.067868    0.02107167 -0.04237733
  0.045113    0.07763367 -0.023825   -0.01223167 -0.024834    0.00498233
  0.09367567  0.01156367 -0.08171733 -0.00808033 -0.02360967  0.018158
  0.040288   -0.05371667  0.01466867 -0.064713   -0.03671033 -0.040814
  0.03923933  0.04472867  0.02500633  0.00970067  0.003613    0.05830767
 -0.010473   -0.00847233 -0.00520533  0.03703933  0.008759   -0.02097633
  0.00471067 -0.00512833 -0.05676467  0.006658    0.01867967 -0.00326733
  0.032311   -0.020348   -0.01410233  0.043098   -0.00092833

In [59]:
# ##################### Build the neural network model #######################
# This script supports three types of models. For each one, we define a
# function that takes a Theano variable representing the input and returns
# the output layer of a neural network model built in Lasagne.

def build_mlp(input_var=None):
    # This creates an MLP of two hidden layers of 800 units each, followed by
    # a softmax output layer of 10 units. It applies 20% dropout to the input
    # data and 50% dropout to the hidden layers.

    # Input layer, specifying the expected input shape of the network
    # (unspecified batchsize, 1 channel, 28 rows and 28 columns) and
    # linking it to the given Theano variable `input_var`, if any:
    l_in = lasagne.layers.InputLayer(shape=(None, 300),
                                     input_var=input_var)

    # Apply 20% dropout to the input data:
    l_in_drop = lasagne.layers.DropoutLayer(l_in, p=0.2)

    # Add a fully-connected layer of 800 units, using the linear rectifier, and
    # initializing weights with Glorot's scheme (which is the default anyway):
    l_hid1 = lasagne.layers.DenseLayer(
            l_in_drop, num_units=800,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())

    # We'll now add dropout of 50%:
    l_hid1_drop = lasagne.layers.DropoutLayer(l_hid1, p=0.5)

    # Another 800-unit layer:
    l_hid2 = lasagne.layers.DenseLayer(
            l_hid1_drop, num_units=800,
            nonlinearity=lasagne.nonlinearities.rectify)

    # 50% dropout again:
    l_hid2_drop = lasagne.layers.DropoutLayer(l_hid2, p=0.5)

    # Finally, we'll add the fully-connected output layer, of 5 softmax units:
    l_out = lasagne.layers.DenseLayer(
            l_hid2_drop, num_units=5,
            nonlinearity=lasagne.nonlinearities.softmax)

    # Each layer is linked to its incoming layer(s), so we only need to pass
    # the output layer to give access to a network in Lasagne:
    return l_out


In [62]:
# ############################# Batch iterator ###############################
# This is just a simple helper function iterating over training data in
# mini-batches of a particular size, optionally in random order. It assumes
# data is available as numpy arrays. For big datasets, you could load numpy
# arrays as memory-mapped files (np.load(..., mmap_mode='r')), or write your
# own custom data iteration function. For small datasets, you can also copy
# them to GPU at once for slightly improved performance. This would involve
# several changes in the main program, though, and is not demonstrated here.

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert inputs.shape[0] == len(targets)
    print (inputs.shape[0])
    if shuffle:
        indices = np.arange(inputs.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, inputs.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [61]:
model='mlp'
num_epochs=500

# Load the dataset
print("Loading data...")
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
y_val = y_val.astype(np.int32)

# Prepare Theano variables for inputs and targets
input_var = T.matrix('inputs')
target_var = T.ivector('targets')

# Create neural network model (depending on first command line parameter)
print("Building model and compiling functions...")
network = build_mlp(input_var)

# Create a loss expression for training, i.e., a scalar objective we want
# to minimize (for our multi-class problem, it is the cross-entropy loss):
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()
# We could add some weight decay as well here, see lasagne.regularization.

# Create update expressions for training, i.e., how to modify the
# parameters at each training step. Here, we'll use Stochastic Gradient
# Descent (SGD) with Nesterov momentum, but Lasagne offers plenty more.
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.01, momentum=0.9)

# Create a loss expression for validation/testing. The crucial difference
# here is that we do a deterministic forward pass through the network,
# disabling dropout layers.
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                        target_var)

test_loss = test_loss.mean()
# As a bonus, also create an expression for the classification accuracy:
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                  dtype=theano.config.floatX)

# Compile a function performing a training step on a mini-batch (by giving
# the updates dictionary) and returning the corresponding training loss:
train_fn = theano.function([input_var, target_var], loss, updates=updates, allow_input_downcast=True)

# Compile a second function computing the validation loss and accuracy:
val_fn = theano.function([input_var, target_var], [test_loss, test_acc], allow_input_downcast=True)

pred_fn = theano.function([input_var], test_prediction)

# Finally, launch the training loop.
print("Starting training...")
# We iterate over epochs:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, 500, shuffle=True):
        inputs, targets = batch
        a = train_fn(inputs, targets) #a is the loss
        train_err += a
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, 500, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

# After training, we compute and print the test error:
test_err = 0
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500, shuffle=False):
    inputs, targets = batch
    err, acc = val_fn(inputs, targets)
    test_err += err
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

# Optionally, you could now dump the network weights to a file like this:
np.savez('model2.npz', *lasagne.layers.get_all_param_values(network))
#
# And load them again later on like this:
# with np.load('model.npz') as f:
#     param_values = [f['arr_%d' % i] for i in range(len(f.files))]
# lasagne.layers.set_all_param_values(network, param_values)

Loading data...
Building model and compiling functions...
Starting training...
Epoch 1 of 500 took 16.216s
  training loss:		1.314607
  validation loss:		1.266746
  validation accuracy:		51.57 %
Epoch 2 of 500 took 14.301s
  training loss:		1.260531
  validation loss:		1.247242
  validation accuracy:		51.57 %
Epoch 3 of 500 took 13.687s
  training loss:		1.239886
  validation loss:		1.224182
  validation accuracy:		51.57 %
Epoch 4 of 500 took 14.322s
  training loss:		1.213231
  validation loss:		1.195559
  validation accuracy:		52.30 %
Epoch 5 of 500 took 14.392s
  training loss:		1.185297
  validation loss:		1.170521
  validation accuracy:		53.66 %
Epoch 6 of 500 took 14.824s
  training loss:		1.160159
  validation loss:		1.141442
  validation accuracy:		54.91 %
Epoch 7 of 500 took 16.773s
  training loss:		1.138290
  validation loss:		1.125817
  validation accuracy:		57.07 %
Epoch 8 of 500 took 14.897s
  training loss:		1.115625
  validation loss:		1.103561
  validation accuracy:		5